## CSC14005 – MACHINE LEARNING
## FINAL PROJECT

### Students
- Le Hoang Sang - 21127158
- Tran Minh Quang - 21127

### Prepare libs 

In [5]:
import fashion_mnist.utils.mnist_reader as mnist_reader
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
import time
import numpy as np

X_train, y_train = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('fashion_mnist/data/fashion', kind='t10k')

In [6]:
X_train = X_train / 255
X_test = X_test / 255
X_train.shape

(60000, 784)

In [7]:
y_train.shape

(60000,)

### Using Linear Kernel

In [8]:
#linear model

# model_linear = SVC(kernel='linear', C = 0.5)
# model_linear.fit(X_train, y_train)

# # predict
# y_pred = model_linear.predict(X_test)

# print(classification_report(y_test, y_pred))

# ----------------------------------------------

# Tạo danh sách các giá trị C bạn muốn thử nghiệm
C_values = [0.1, 0.5, 1.0, 2.0, 5.0]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

for C in C_values:
    # Bắt đầu đo thời gian
    start_time = time.time()

    # Tạo và huấn luyện mô hình với giá trị C hiện tại
    model_linear = SVC(kernel='linear', C=C)
    model_linear.fit(X_train, y_train)``

    # Đo thời gian huấn luyện
    training_time = time.time() - start_time

    # Tính độ lỗi trên tập huấn luyện
    train_error = 1 - model_linear.score(X_train, y_train)

    # Tính độ lỗi trên tập validation sử dụng cross-validation
    val_error = 1 - np.mean(cross_val_score(model_linear, X_train, y_train, cv=5))

    # In ra kết quả
    print(f"C={C} - Training Error: {train_error:.4f}, Validation Error: {val_error:.4f}, Training Time: {training_time:.2f} seconds")

C=0.1 - Training Error: 0.1095, Validation Error: 0.1361, Training Time: 179.99 seconds


### Comment results

### Using RBF Kernel

In [ ]:
# RBF model

# model_rbf = SVC(kernel='rbf', C = 0.5, gamma = 'auto')
# model_rbf.fit(X_train, y_train)

# # predict
# y_pred = model_rbf.predict(X_test)

# print(classification_report(y_test, y_pred))

# ------------------------------------------------

# Tạo danh sách các giá trị C và gamma bạn muốn thử nghiệm
C_values = [0.1, 0.5, 1.0, 2.0, 5.0]
gamma_values = ['auto', 0.1, 0.5, 1.0, 2.0]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

for C in C_values:
    for gamma in gamma_values:
        # Bắt đầu đo thời gian
        start_time = time.time()

        # Tạo và huấn luyện mô hình với giá trị C và gamma hiện tại
        model_rbf = SVC(kernel='rbf', C=C, gamma=gamma)
        model_rbf.fit(X_train, y_train)

        # Đo thời gian huấn luyện
        training_time = time.time() - start_time

        # Tính độ lỗi trên tập huấn luyện
        train_error = 1 - model_rbf.score(X_train, y_train)

        # Tính độ lỗi trên tập validation sử dụng cross-validation
        val_error = 1 - np.mean(cross_val_score(model_rbf, X_train, y_train, cv=5))

        # In ra kết quả
        print(f"C={C}, gamma={gamma} - Training Error: {train_error:.4f}, Validation Error: {val_error:.4f}, Training Time: {training_time:.2f} seconds")

### Comment results

### Review SVM

### References